In [1]:
import agate

In [2]:
smelters = agate.Table.from_csv('Smelters_Mines.csv')
print smelters

|---------------+---------------|
|  column_names | column_types  |
|---------------+---------------|
|  OBJECTID *   | Number        |
|  Shape *      | Text          |
|  ID           | Number        |
|  SEQUENCE     | Number        |
|  ADD_         | Text          |
|  ELEVM        | Number        |
|  ELEVF        | Number        |
|  ZONE         | Date          |
|  NORTHING     | Number        |
|  EASTING      | Number        |
|  TWODEG       | Text          |
|  X_1_2__QUA   | Text          |
|  FIPS         | Number        |
|  COUNTY       | Text          |
|  TWP          | Date          |
|  RNG          | TimeDelta     |
|  SEC          | Number        |
|  QUARTER      | Text          |
|  PROJECTED    | Boolean       |
|  LAND_GRANT   | Number        |
|  PROPNAME     | Text          |
|  TYPEOPR      | Text          |
|  SURFACE      | Text          |
|  UNDERGROUN   | Text          |
|  SMELTER      | Text          |
|  TAILINGS     | Boolean       |
|  WASTE_ROCK 

In [7]:
surface = smelters.aggregate(agate.Count('TYPEOPR', 'Surface; Underground'))
print surface

166


In [35]:
prospect = smelters.aggregate(agate.Count('STATUS', 'Prospect'))
print prospect

185


In [8]:
past = smelters.aggregate(agate.Count('STATUS', 'Past Producer'))
print past

1574


In [9]:
cole = smelters.aggregate(agate.Count('COUNTY', 'Cole'))
print cole

206


In [37]:
num_without_owner = smelters.aggregate(agate.Count('OWNER', None))
print(num_without_owner)

1418


In [11]:
print smelters.columns['COMMODITY']

<agate.Column: (u'Zinc; Lead', u'Zinc-sulfide; Barium', u'Lead-sulfide; Barium', u'Lead-sulfide', u'Barium; Lead', ...)>


In [39]:
by_commodity = smelters.group_by('COMMODITY')
commodity_totals = by_commodity.aggregate([
    ('count', agate.Count())
])
sorted_totals = commodity_totals.order_by('count', reverse=True)
sorted_totals.print_table(max_rows=10)

|-----------------------+--------|
|  COMMODITY            | count  |
|-----------------------+--------|
|  Barium               |   571  |
|  Lead                 |   439  |
|  Lead; Zinc           |   145  |
|  Lead; Barium         |    87  |
|  Barium; Lead         |    77  |
|  Lead-sulfide         |    65  |
|  Zinc                 |    46  |
|  Lead?                |    44  |
|  Lead?; Barium?       |    40  |
|  Lead-sulfide; Barium |    29  |
|  ...                  |   ...  |
|-----------------------+--------|


In [15]:
by_county = smelters.group_by('COUNTY')
county_totals = by_county.aggregate([
    ('count', agate.Count())
])

sorted_totals = county_totals.order_by('count', reverse=True)
sorted_totals.print_table(max_rows=5)

|-----------+--------|
|  COUNTY   | count  |
|-----------+--------|
|  Miller   |   669  |
|  Morgan   |   596  |
|  Moniteau |   244  |
|  Cole     |   206  |
|  Camden   |    90  |
|  ...      |   ...  |
|-----------+--------|


In [22]:
by_operator = smelters.group_by('OPERATOR')
operator_totals = by_operator.aggregate([
    ('count', agate.Count())
])
sorted_totals = operator_totals.order_by('count', reverse=True)
sorted_totals.print_table(max_rows=6)

|-----------------------+--------|
|  OPERATOR             | count  |
|-----------------------+--------|
|                       | 1,417  |
|  O.S. Reavis          |    17  |
|  Ozark Mining and ... |     9  |
|  John White           |     7  |
|  Campbell Brothers    |     6  |
|  Union Electric       |     6  |
|  ...                  |   ...  |
|-----------------------+--------|


In [58]:
binned_depth = smelters.bins('DEPTH', 20, 0, 300)
binned_depth.print_bars('DEPTH', 'Count', width=80)

DEPTH       Count
[0 - 15)    1,597 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░            
[15 - 30)     110 ▓░░░                                                          
[30 - 45)      87 ▓░░░                                                          
[45 - 60)      40 ▓░                                                            
[60 - 75)      38 ▓░                                                            
[75 - 90)      47 ▓░                                                            
[90 - 105)     34 ▓░                                                            
[105 - 120)     9 ▓                                                             
[120 - 135)    16 ▓                                                             
[135 - 150)    12 ▓                                                             
[150 - 165)    12 ▓                                                             
[165 - 180)     6 ▓                                                             
[180 - 195

In [36]:
smelters.aggregate([
    agate.Mean('DEPTH')
])

(Decimal('13.70371287128712871287128713'),)

In [70]:
with_commodity_in_county = smelters.compute([
    ('years_in_prison', agate.Change('convicted', 'exonerated'))
])

county_totals = with_years_in_prison.group_by('state')

medians = state_totals.aggregate([
    ('count', agate.Count()),
    ('median_years_in_prison', agate.Median('years_in_prison'))
])

sorted_medians = medians.order_by('median_years_in_prison', reverse=True)

sorted_medians.print_table(max_rows=5)

|---------------+--------|
|  COMMODITY    | count  |
|---------------+--------|
|  Barium       |   571  |
|  Lead         |   439  |
|  Lead; Zinc   |   145  |
|  Lead; Barium |    87  |
|  Barium; Lead |    77  |
|  Lead-sulfide |    65  |
|  ...          |   ...  |
|---------------+--------|


In [78]:
operators_by_county = smelters.group_by("COUNTY").group_by('OPERATOR')

In [82]:
new_table = smelters.distinct('OPERATOR')